<a href="https://colab.research.google.com/github/sqhang/githubTest/blob/main/neatscribe_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 0. Install and import libraries

Need to use GPU runtime for Whisper to work

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import json

In [3]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

import whisper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-61g31hy5
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-61g31hy5
  Resolved https://github.com/openai/whisper.git to commit c09a7ae299c4c34c5839a76380ae407e7d785914
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://

In [5]:
! pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import jiwer
from whisper.normalizers import EnglishTextNormalizer

Run this block only if we want to use Google Speech2Text

In [7]:
!pip install google-cloud-speech
!pip install google-auth google-auth-oauthlib google-auth-httplib2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Part 1. Merge Audios in dev folder

In [ ]:
import os
from pydub import AudioSegment

def merge_sentences_in_paragraphs(input_folder, output_folder, target_format):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through the input files and sort them based on sentence index
    files = os.listdir(input_folder)
    files.sort(key=lambda f: tuple(map(int, f[:-4].split('_'))))

    # Merge sentences in paragraphs
    paragraphs = {}
    for file_name in files:
        # Check if the file is a WAV file
        if file_name.endswith(".wav"):
            # Parse the file name to extract the indices
            file_base = file_name[:-4]
            article_idx, paragraph_idx, sentence_idx = map(int, file_base.split('_'))

            # Load the input file and convert it to the target format
            file_path = os.path.join(input_folder, file_name)
            input_audio = AudioSegment.from_wav(file_path)
            input_audio = input_audio.set_channels(target_format["channels"])
            input_audio = input_audio.set_frame_rate(target_format["frame_rate"])
            input_audio = input_audio.set_sample_width(target_format["sample_width"])

            # Merge the input file with the corresponding paragraph
            key = (article_idx, paragraph_idx)
            if key not in paragraphs:
                paragraphs[key] = input_audio
            else:
                paragraphs[key] = paragraphs[key] + input_audio

    # Save the merged audio files to the output folder
    for (article_idx, paragraph_idx), audio in paragraphs.items():
        output_file_name = f"{article_idx}_{paragraph_idx}_merged.wav"
        output_file_path = os.path.join(output_folder, output_file_name)
        audio.export(output_file_path, format="wav")


In [ ]:
!unzip /content/drive/MyDrive/SpokenSQuAD/dev_wav.zip

In [ ]:
# Input folder containing the WAV files
input_folder = "dev_wav"

# Output folder where the merged WAV files will be saved
output_folder = "/content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_44.1khz_2"

# Target format for the input files
target_format = {
    "channels": 1,        # Mono
    "frame_rate": 44100,  # 44.1 kHz
    "sample_width": 2     # 16 bits per sample
}

# Merge the sentences in the paragraphs and save the results to the output folder
merge_sentences_in_paragraphs(input_folder, output_folder, target_format)

In [ ]:
# Input folder containing the WAV files
input_folder = "dev_wav"

# Output folder where the merged WAV files will be saved
output_folder_lower = "/content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_4khz_1"

target_format_lower = {
    "channels": 1,        # Stereo
    "frame_rate": 4000,  # 48 kHz
    "sample_width": 1     # 24 bits per sample
}

# Merge the sentences in the paragraphs and save the results to the output folder
merge_sentences_in_paragraphs(input_folder, output_folder_lower, target_format_lower)

In [ ]:
! ls -1 /content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_4khz_1 | wc -l

2067


In [ ]:
! ls -1 /content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_44.1khz_2 | wc -l

2067


In [ ]:
! du -sh /content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_44.1khz_2

11G	/content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_44.1khz_2


In [ ]:
! du -sh /content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_4khz_1

487M	/content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_4khz_1


Sanity check: compare the list and see the diff!

In [ ]:
import os

list1 = os.listdir("/content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_4khz_1")
list2 = os.listdir("/content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_44.1khz_2")
print(list1)
print(list2)

extra_list = [elem for elem in list1 if (elem not in list2)]
extra_list

['0_0_merged.wav', '0_1_merged.wav', '0_2_merged.wav', '0_3_merged.wav', '0_4_merged.wav', '0_5_merged.wav', '0_6_merged.wav', '0_7_merged.wav', '0_8_merged.wav', '0_9_merged.wav', '0_10_merged.wav', '0_11_merged.wav', '0_12_merged.wav', '0_13_merged.wav', '0_14_merged.wav', '0_15_merged.wav', '0_16_merged.wav', '0_17_merged.wav', '0_18_merged.wav', '0_19_merged.wav', '0_20_merged.wav', '0_21_merged.wav', '0_22_merged.wav', '0_23_merged.wav', '0_24_merged.wav', '0_25_merged.wav', '0_26_merged.wav', '0_27_merged.wav', '0_28_merged.wav', '0_29_merged.wav', '0_30_merged.wav', '0_31_merged.wav', '0_32_merged.wav', '0_33_merged.wav', '0_34_merged.wav', '0_35_merged.wav', '0_36_merged.wav', '0_37_merged.wav', '0_38_merged.wav', '0_39_merged.wav', '0_40_merged.wav', '0_41_merged.wav', '0_42_merged.wav', '0_43_merged.wav', '0_44_merged.wav', '0_45_merged.wav', '0_46_merged.wav', '0_47_merged.wav', '0_48_merged.wav', '0_49_merged.wav', '0_50_merged.wav', '0_51_merged.wav', '0_52_merged.wav', '0

# Part 2. Get Ground Truth Data for Spoken SQuAD Speech (need to add helper routines to extract QA pairs) 

Read from json files

Train: https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json \n
Validation: https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json

In [22]:
def get_dev_context(article_idx, para_idx):
  f = open('/content/drive/MyDrive/SpokenSQuAD/dev-v1.1.json')
  data = json.load(f)
  f.close()
  return data["data"][article_idx]["paragraphs"][para_idx]["context"]


In [23]:
get_dev_context(0, 0)

'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.'

# Part 3. Convert Speech to Text Whisper

In [34]:
import whisper

medium_model = whisper.load_model("medium")

def transcribe_audio_whisper(whisper_model, audio_pathname):
  result = whisper_model.transcribe(audio_pathname)
  return result["text"]

In [35]:
whisper_sol_3_54 = transcribe_audio_whisper(medium_model, "/content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_4khz_1/3_54_merged.wav")
whisper_sol_3_54

" There have been subsequent claims by Tesla biographers that Edison and Tesla were the original recipients and that neither was given the award because of their animosity toward each other, that OOPS sought to minimize the other's achievements and right to win the award, that both refused ever to accept the award if the other received it first, that both rejected any possibility of sharing it, and even that a wealthy Edison refused it to coup Tesla from getting the $20.00 prize money."

In [36]:
whisper_sol_2_0 = transcribe_audio_whisper(medium_model, "/content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_4khz_1/2_0_merged.wav")
whisper_sol_2_0

' The Normans, Norman, Normans, French, Normans, Latin, Normannes, were the people who in the 10th and 11th centuries stood their noon to Normandou. They were descended from Norse. Norman comes from Norsemen. Woters and pirates from Denmark, Iceland and Norway who, under their looter Rolo, agreed to swear filth to King Charles II of West Francia through generations of assimilation and mixing with the native Frankish and Roman Gauls. Populations. Their descendants would gradually merge with the Carolingian-dosed cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'

# Part 4. Convert Speech to Text Wav2vec

In [12]:
!pip install transformers torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.3 MB/s eta 0:00:00


In [19]:
import torch
import torchaudio
import warnings
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Filter out warning messages
warnings.filterwarnings("ignore", module="torchaudio")

# Load the pre-trained Wav2Vec2 model and the processor
model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

def transcribe_audio_wav2vec(audio_pathname):
    # Load the audio file
    audio_input, sample_rate = torchaudio.load(audio_pathname)

    # Check if the audio has the correct sample rate (16 kHz for the wav2vec2-base-960h model)
    if sample_rate != 16000:
        audio_input = torchaudio.transforms.Resample(sample_rate, 16000)(audio_input)
    
    # Preprocess the audio data
    input_values = processor(audio_input.squeeze().numpy(), return_tensors="pt", sampling_rate=16000).input_values

    # Perform zero-shot prediction with the Wav2Vec2 model
    with torch.no_grad():
        logits = model(input_values).logits

    # Decode the logits into text
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    return transcription


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Transcribe the audio file
file_path = "/content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_44.1khz_2/3_54_merged.wav"
transcription_output = transcribe_audio_wav2vec(file_path)

print(transcription_output)

THERE HAVE BEEN SUBSEQUENT CLAIMS BY TESTLA BIOGRAPHERS THAT EDISON AND TESTLA WERE THE ORIGINAL RECIPIENTS AND THAT NEITHER WAS GIVEN THE AWARD BECAUSE OF THEIR ANIMOSITY TOWARD EACH OTHER THAT EACH SOUGHT TO MINIMIZE THE OTHER'S ACHIEVEMENTS AND RIGHT TO WIN THE AWARD THAT BOTH REFUSED EVER TO ACCEPT THE AWARD IF THE OTHER RECEIVED IT FIRST THAT BOTH REJECTED ANY POSSIBILITY OF SHARING IT AND EVEN THAT A WEALTHY EDISON REFUSED IT TO KEEP TESTLA FROM GETTING THE TWENTY DOLLARS OH O O PRIZE MONEY TWO HUNDRED FORTY FIVE


# Part 5. Google Speech2Text Conversion

In [ ]:
!pip install google-cloud-speech
!pip install google-auth google-auth-oauthlib google-auth-httplib2

In [9]:
import io
import os
from google.cloud import speech_v1p1beta1 as speech
from google.oauth2 import service_account

def google_transcribe_audio_file(audio_file_path, credentials_file_path):
    # Set up the client using the service account key
    credentials = service_account.Credentials.from_service_account_file(credentials_file_path)
    client = speech.SpeechClient(credentials=credentials)

    # Read the audio file
    with io.open(audio_file_path, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)

    # Set up the configuration
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=44100,
        language_code="en-US",
    )

    # Make the API request
    response = client.recognize(config=config, audio=audio)

    # Combine the transcriptions
    transcript = " ".join([result.alternatives[0].transcript for result in response.results])

    return transcript

In [11]:
# Replace these paths with the path to your audio file and your service account key JSON file
audio_file_path = "/content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_44.1khz_2/3_54_merged.wav"
credentials_file_path = "/content/drive/MyDrive/SpokenSQuAD/neatscribe-384719-800219ebfd62.json"

transcript = google_transcribe_audio_file(audio_file_path, credentials_file_path)
print("Transcript:", transcript)

Transcript: there have been subsequent claims by Tesla biographers that Edison and Tesla where the original recipients and that neither was given the award because of their animosity toward each other that each. To minimize the others achieve mints and write to win the award that both refused ever to accept the award if the other received it first don't reject it any possibility of sharing it and not a wealthy Edison refused it to keep Tesla from getting the $20 Olo prize money 245


# Part 6. Evaluate WER on transcripts

In [27]:
ref_sol_3_54 = get_dev_context(3, 54)
ref_sol_3_54

"There have been subsequent claims by Tesla biographers that Edison and Tesla were the original recipients and that neither was given the award because of their animosity toward each other; that each sought to minimize the other's achievements and right to win the award; that both refused ever to accept the award if the other received it first; that both rejected any possibility of sharing it; and even that a wealthy Edison refused it to keep Tesla from getting the $20,000 prize money.:245"

In [28]:
ref_sol_2_0 = get_dev_context(2, 0)
ref_sol_2_0

'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'

This WER evaluation scripts were adapted from the Whisper official website!

In [26]:
import jiwer
from whisper.normalizers import EnglishTextNormalizer

normalizer = EnglishTextNormalizer()

In [29]:
def wer_eval(gen_t, ref_t):
  return jiwer.wer(normalizer(gen_t), normalizer(ref_t))

In [37]:
wer_eval(whisper_sol_3_54, ref_sol_3_54)

0.047619047619047616

In [38]:
wer_eval(whisper_sol_2_0, ref_sol_2_0)

0.21621621621621623

In [24]:
wav2vec_sol_2_0_high = transcribe_audio_wav2vec("/content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_44.1khz_2/2_0_merged.wav")

In [32]:
wav2vec_sol_2_0_low = transcribe_audio_wav2vec("/content/drive/MyDrive/SpokenSQuAD/output_dev_wav_1_4khz_1/2_0_merged.wav")

In [39]:
wav2vec_sol_2_0_low

'THE NORMAN NORMAN NORMAN FREN NORMAN LATAN NORMAMON WERE THE POVAL WHOIN THE CENTAND ELEVEN SENTE TO THE NON TO NORMANDO A RUDEN AND FRAN THE WERE DESCNDED FROM NORTH NORMAN COM FROM NORTHMAN WEODERS AND PIRATE FROM DENMARK ISLAN AND NORWA VO UNDER THEIR LDER ROLO A GRU DEWARE ILLTDO THE TON FALLVOM THEF WES FANA THROG DENALOSANS OF THE SIMILOSIN AND NITON WITH THE NOTOF SENTETS AND NORMAN GALE POPULOTAN TERE DESENDANTS WIT GRATUE MURGS WITH THE SEROLANSAN TOS CALTERS OF WEST FANTA THE DEFOCULTRA AND ETNET IDENDEDOF THE NORMANS THE MURDS THE NISILY IN THE FIRT HALFOF TU SEN SENTERU UNTHE CONTOSWHO VALL OVER TE OF FUDUM CENTRU'

In [33]:
wer_eval(normalizer(wav2vec_sol_2_0_low), normalizer(ref_sol_2_0))

0.7927927927927928

In [30]:
wer_eval(normalizer(wav2vec_sol_2_0_high), normalizer(ref_sol_2_0))

0.15789473684210525

# Part 7. Convert Transcripts and Evaluate WER for each articleIdx_paraIdx (ongoing)

# Scratch Block

In [ ]:
import re
from jiwer import wer

def normalize_contractions(transcript):
    contractions_dict = {
        "won't": "will not",
        "can't": "cannot",
        "I'm" : "I am",
        "i'm" : "i am",
        "You're": "You are",
        "you're": "you are"
        # Add more contractions and their normalized forms as needed
    }
    
    for contraction, normalized_form in contractions_dict.items():
        transcript = transcript.replace(contraction, normalized_form)
        
    return transcript

def preprocess_transcript(transcript):
    # Remove special characters, except for apostrophes (since contractions have them)
    transcript = re.sub(r"[^a-zA-Z\s']", "", transcript)

    # Normalize contractions
    transcript = normalize_contractions(transcript)

    # Remove filler words (you can add more filler words to the list as needed)
    filler_words = ["um", "uh", "like"]
    words = transcript.split()
    words = [word for word in words if word.lower() not in filler_words]
    transcript = ' '.join(words)

    # Remove numbers (keep the text representation of numbers)
    transcript = re.sub(r'\d+', '', transcript)

    # Lowercase and return the preprocessed transcript
    return transcript.lower()

def compute_wer(true_transcript, predicted_transcript):
    # Preprocess the transcripts
    true_transcript = preprocess_transcript(true_transcript)
    predicted_transcript = preprocess_transcript(predicted_transcript)

    return wer(true_transcript, predicted_transcript)


In [ ]:
import openai
import tensorflow_datasets as tfds
from jiwer import wer
import numpy as np
import requests

# Load the TED-LIUM dataset
tedlium_dataset = tfds.load('tedlium')
test_data = tedlium_dataset['test']

# Initialize the OpenAI API
openai.api_key = "your_openai_api_key"

# Set up the Whisper ASR API endpoint
whisper_url = "https://api.openai.com/v1/whisper/asr"

# Function to transcribe audio using the Whisper ASR API
def transcribe_audio(audio):
    response = requests.post(
        whisper_url,
        headers={"Authorization": f"Bearer {openai.api_key}"},
        files={"file": audio},
    )
    response.raise_for_status()
    return response.json()["transcript"]

# Function to compute Word Error Rate
def compute_wer(true_transcript, predicted_transcript):
    return wer(true_transcript, predicted_transcript)

# Set up variables for WER calculation
num_samples = 100  # You can adjust this number as needed
wer_values = []

# Iterate through the test dataset and compute WER
for i, sample in enumerate(test_data.take(num_samples)):
    audio = sample['audio'].numpy().tobytes()
    true_transcript = sample['transcript'].numpy().decode('utf-8')
    
    try:
        predicted_transcript = transcribe_audio(audio)
        sample_wer = compute_wer(true_transcript, predicted_transcript)
        wer_values.append(sample_wer)
        print(f"Sample {i + 1}: WER = {sample_wer:.2f}")
    except Exception as e:
        print(f"Error in sample {i + 1}: {e}")

# Calculate the average WER for the test dataset
average_wer = np.mean(wer_values)
print(f"Average WER for {num_samples} samples: {average_wer:.2f}")

In [ ]:
import subprocess

# The command to transcribe the audio using the whisper ASR tool without timestamps
cmd = "whisper /content/3_54_merged.wav --model medium.en --no-timestamps"

# Run the command and capture the output
transcription_output = subprocess.check_output(cmd, shell=True, text=True)

# The transcribed text without time annotations is now stored in the transcription_output variable
print(transcription_output)

In [ ]:
# This hasn't really run due to long running time!

# # Input folder containing the WAV files
# input_folder = "dev_wav"

# # Output folder where the merged WAV files will be saved
# output_folder_higher = "/content/drive/MyDrive/SpokenSQuAD/output_wav_2_48khz_3"

# target_format_higher = {
#     "channels": 2,        # Stereo
#     "frame_rate": 48000,  # 48 kHz
#     "sample_width": 3     # 24 bits per sample
# }

# # Merge the sentences in the paragraphs and save the results to the output folder
# merge_sentences_in_paragraphs(input_folder, output_folder_higher, target_format_higher)

Use Tensorflow libraries

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 24.0 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow-datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow_datasets as tfds

squad_v1, info = tfds.load("squad/v1.1", with_info=True, split=["train", "validation"])

train_data, validation_data = squad_v1

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/squad/v1.1/3.0.0.incompleteWJD629/squad-train.tfrecord*...:   0%|         …

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/squad/v1.1/3.0.0.incompleteWJD629/squad-validation.tfrecord*...:   0%|    …

Dataset squad downloaded and prepared to /root/tensorflow_datasets/squad/v1.1/3.0.0. Subsequent calls will reuse this data.


In [ ]:
train_data

<_PrefetchDataset element_spec={'answers': {'answer_start': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'text': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, 'context': TensorSpec(shape=(), dtype=tf.string, name=None), 'id': TensorSpec(shape=(), dtype=tf.string, name=None), 'question': TensorSpec(shape=(), dtype=tf.string, name=None), 'title': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [ ]:
# Iterate over the validation_data
counter = 0
for example in validation_data:
    id = example["id"].numpy().decode('utf-8')
    context = example["context"].numpy().decode("utf-8")
    question = example["question"].numpy().decode("utf-8")
    answer_text = example["answers"]["text"][0].numpy().decode("utf-8")
    answer_start = example["answers"]["answer_start"][0].numpy()
    
    print("id:", id)
    print("Context:", context)
    print("Question:", question)
    print("Answer:", answer_text)
    print("Answer Start Position:", answer_start)
    print("\n---\n")
    counter += 1
    if counter > 100:
      break

In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json

--2023-04-23 20:05:11--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘train-v1.1.json’

train-v1.1.json     100%[===================>]  28.88M   147MB/s    in 0.2s    

2023-04-23 20:05:13 (147 MB/s) - ‘train-v1.1.json’ saved [30288272/30288272]



In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json

--2023-04-23 20:05:48--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘dev-v1.1.json’

dev-v1.1.json       100%[===================>]   4.63M  --.-KB/s    in 0.09s   

2023-04-23 20:05:50 (53.4 MB/s) - ‘dev-v1.1.json’ saved [4854279/4854279]



In [ ]:
train_data

In [ ]:
from pydub import AudioSegment
import os

def convert_frame_rate(input_folder, output_folder, target_frame_rate):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through the input files
    for file_name in os.listdir(input_folder):
        # Check if the file is a WAV file
        if file_name.endswith(".wav"):
            # Load the input file
            file_path = os.path.join(input_folder, file_name)
            input_audio = AudioSegment.from_wav(file_path)

            # Convert the frame rate
            converted_audio = input_audio.set_frame_rate(target_frame_rate)

            # Save the converted audio to the output folder
            output_file_path = os.path.join(output_folder, file_name)
            converted_audio.export(output_file_path, format="wav")

# Source folder containing the 44.1 kHz merged WAV files
source_folder = "output_wav_44_1"

# Destination folder where the 48 kHz converted WAV files will be saved
destination_folder = "output_wav_48"

# Target frame rate (48 kHz)
target_frame_rate = 48000

# Convert the frame rate of the merged files and save the results to the destination folder
convert_frame_rate(source_folder, destination_folder, target_frame_rate)

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-sdvbfj5w
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-sdvbfj5w
  Resolved https://github.com/openai/whisper.git to commit c09a7ae299c4c34c5839a76380ae407e7d785914
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798038 sha256=85fca8d3349c1e6f1783c908fb87d083205b6ff33e4a3836f36a70b18fa1aaf2
  Stored in directory: /tmp/pip-ephem-wheel-cache-jh5x63m3/wheels/fe/03/29/e7919208d11b4ab32972cb448bb84a9a675d92cd52c9a48341
Successfully built openai-whisper
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-

In [ ]:
!whisper "/content/3_54_merged.wav" --model medium.en

100%|█████████████████████████████████████| 1.42G/1.42G [01:09<00:00, 22.0MiB/s]
/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Traceback (most recent call last):
  File "/usr/local/bin/whisper", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py", line 448, in cli
    result = transcribe(model, audio_path, temperature=temperature, **args)
  File "/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py", line 229, in transcribe
    result: DecodingResult = decode_with_fallback(mel_segment)
  File "/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py", line 164, in decode_with_fallback
    decode_result = model.decode(segment, options)
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs

In [ ]:
!wget -P /content/drive/MyDrive/ http://speech.ee.ntu.edu.tw/~chiahsuan/Spoken-SQuAD/Spoken-SQuAD_audio.zip

--2023-04-22 22:08:29--  http://speech.ee.ntu.edu.tw/~chiahsuan/Spoken-SQuAD/Spoken-SQuAD_audio.zip
Resolving speech.ee.ntu.edu.tw (speech.ee.ntu.edu.tw)... 140.112.21.8
Connecting to speech.ee.ntu.edu.tw (speech.ee.ntu.edu.tw)|140.112.21.8|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46069259699 (43G) [application/zip]
Saving to: ‘/content/drive/MyDrive/Spoken-SQuAD_audio.zip’

Spoken-SQuAD_audio. 100%[===================>]  42.91G  12.0MB/s    in 66m 11s 

2023-04-22 23:14:41 (11.1 MB/s) - ‘/content/drive/MyDrive/Spoken-SQuAD_audio.zip’ saved [46069259699/46069259699]



In [ ]:
!unzip /content/drive/MyDrive/SpokenSQuAD/Spoken-SQuAD_audio.zip

Archive:  /content/drive/MyDrive/SpokenSQuAD/Spoken-SQuAD_audio.zip
   creating: Spoken-SQuAD_audio/
 extracting: Spoken-SQuAD_audio/dev_wav.zip  
 extracting: Spoken-SQuAD_audio/train_wav.zip  


In [ ]:
!rm -rf /content/speech.ee.ntu.edu.tw

In [ ]:
print(1)

1


In [ ]:
!ls /content/drive/MyDrive/SpokenSQuAD

dev_wav.zip  Spoken-SQuAD_audio.zip  train_wav.zip


In [ ]:
!unzip /content/Spoken-SQuAD_audio/dev_wav.zip

unzip:  cannot find or open /content/Spoken-SQuAD_audio/dev_wav.zip, /content/Spoken-SQuAD_audio/dev_wav.zip.zip or /content/Spoken-SQuAD_audio/dev_wav.zip.ZIP.


In [ ]:
!unzip /content/drive/MyDrive/SpokenSQuAD/dev_wav.zip

In [ ]:
!ls !unzip /content/dev_wav

In [ ]:
!unzip /content/drive/MyDrive/SpokenSQuAD/train_wav.zip

In [ ]:
!ls /content/train_wav | head -n 50

In [ ]:
! ls -1 /content/dev_wav | wc -l

10578


In [ ]:
! ls -1 /content/train_wav | wc -l

94253
